In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [2]:
!pip install torch
!pip install datasets transformers
!pip install sklearn


     |████████████████████████████████| 325 kB 30.6 MB/s 
     |████████████████████████████████| 4.0 MB 65.8 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 136 kB 77.8 MB/s 
     |████████████████████████████████| 212 kB 76.5 MB/s 
     |████████████████████████████████| 1.1 MB 66.1 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
     |████████████████████████████████| 6.6 MB 63.0 MB/s 
     |████████████████████████████████| 895 kB 54.1 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 68.3 MB/s 
     |████████████████████████████████| 144 kB 76.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstallin

In [3]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizerFast
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

In [4]:
checkpoint = "distilbert-base-uncased"

In [6]:
import pandas as pd
dataset1 =pd.read_csv("https://github.com/mohdwasi03/NLP/blob/main/boolq_train.csv",nrows=100, header=None)
dataset2 =pd.read_csv("https://github.com/mohdwasi03/NLP/blob/main/boolq_val.csv",nrows=100, header=None)

dataset1.to_csv('train.csv', header=None, )
dataset2.to_csv('validate.csv', header=None)
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'validate.csv'})



ParserError: ignored

In [ ]:

dataset = load_dataset('csv', data_files={'train': '/content/drive/MyDrive/nlp_project/train.csv', 'validation': '/content/drive/MyDrive/nlp_project/validate.csv'})

In [ ]:
dataset['train'][1]

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    encoded = tokenizer(example["question"], example["passage"], truncation=True)
    encoded["labels"] = [int(a) for a in example["answer"]]
    return encoded
   



tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
args = TrainingArguments("roberta-booql",
per_device_train_batch_size=16,
learning_rate=1e-3,
num_train_epochs=3)

In [ ]:
trainer = Trainer(model,
args,
train_dataset=tokenized_datasets["train"],
eval_dataset=tokenized_datasets["validation"],
data_collator=data_collator,
tokenizer=tokenizer,)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
y_pred = predictions.predictions.argmax(-1)
labels = predictions.label_ids

In [ ]:
print(tokenized_datasets["validation"],labels)

In [ ]:
print(tokenized_datasets["validation"]["passage"][0])
print(tokenized_datasets["validation"]["question"][0])
print(tokenized_datasets["validation"]["answer"][0])
print(tokenized_datasets["validation"]["title"][0])


In [ ]:
metric = load_metric("accuracy")

In [ ]:
metric.compute(predictions=y_pred, references=predictions.label_ids)